In [1]:
import os
os.getcwd()

'/home/yes24/capstone/추천알고리즘/preprocessed'

In [2]:
os.chdir('/home/yes24/models/research/deep_contextual_bandits')

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import tensorflow as tf
import time
from absl import app
from absl import flags
import sys

In [4]:
from bandits.core.contextual_bandit import run_contextual_bandit
from bandits.algorithms.linear_full_posterior_sampling import LinearFullPosteriorSampling

In [5]:
from bandits.data.data_sampler import sample_census_data
from bandits.data.data_sampler import one_hot,safe_std,remove_underrepresented_classes,classification_to_bandit_problem

In [6]:
def display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, name):
    """Displays summary statistics of the performance of each algorithm."""

    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print('{} bandit completed after {} seconds.'.format(
        name, time.time() - t_init))
    print('---------------------------------------------------')

    performance_pairs = []
    for j, a in enumerate(algos):
        performance_pairs.append((a.name, np.sum(h_rewards[:, j])))
    performance_pairs = sorted(performance_pairs,
                             key=lambda elt: elt[1],
                             reverse=True)
    for i, (name, reward) in enumerate(performance_pairs):
        print('{:3}) {:20}| \t \t total reward = {:10}.'.format(i, name, reward))

    print('---------------------------------------------------')
    print('Optimal total reward = {}.'.format(np.sum(opt_rewards)))
    print('Frequency of optimal actions (action, frequency):')
    print([[elt, list(opt_actions).count(elt)] for elt in set(opt_actions)])
    print('---------------------------------------------------')
    print('---------------------------------------------------')



In [7]:
base_route = os.getcwd()
data_route = 'contextual_bandits/datasets'


flags.DEFINE_string('logdir', '/tmp/bandits/', 'Base directory to save output')
flags.DEFINE_string(
    'census_data',
    os.path.join(base_route, data_route, 'USCensus1990.data.txt'),
    'Directory where Census data is stored.')
flags.DEFINE_string('f', '', 'kernel')
FLAGS = flags.FLAGS
FLAGS.set_default('alsologtostderr', True)
FLAGS(sys.argv)

['/home/yes24/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py']

In [8]:
num_contexts=2000

In [9]:
file_name = FLAGS.census_data
num_actions = 9
num_contexts = min(150000, num_contexts)
sampled_vals = sample_census_data(file_name, num_contexts,
                                      shuffle_rows=True)
contexts, rewards, (opt_rewards, opt_actions) = sampled_vals
dataset = np.hstack((contexts, rewards))
context_dim = contexts.shape[1]

In [15]:
context_dim

67

In [18]:
len(contexts)

2458285

In [12]:
len(dataset[0])

76

In [16]:
dataset[0]

array([ 5.60586769e-01, -8.16347592e-01, -3.38562341e-01, -1.79964920e-01,
       -2.98016868e-01,  2.13931918e+00,  9.11708112e-01,  6.72682954e-01,
        6.32357200e-01,  1.21596164e+00, -1.37949966e-01, -9.75311200e-02,
       -1.55379379e-01,  1.40039746e+00,  9.16240022e-01, -2.65393766e-01,
        1.00767472e+00, -2.28285518e-01, -1.04611137e-01,  2.01306479e+00,
       -4.02761499e-01, -1.85116882e-01, -2.66526942e-01, -2.08169478e-01,
        1.62169262e+00, -1.44685093e-01, -1.23277291e+00, -6.16638705e-01,
       -4.79199980e-01, -1.02765454e-01,  7.05498560e-02,  7.05668240e-01,
        1.07348489e+00,  5.77057393e-01, -3.88827893e-02,  5.79902000e-01,
       -2.89975491e-01,  3.97019742e-01, -1.70656214e-01,  1.17693809e+00,
        9.77633623e-01, -6.69015411e-01, -1.04119228e-01, -5.42225044e-01,
        3.49376720e+00, -5.06598665e-01, -5.57212397e-01,  7.65372651e-01,
        4.39704639e-01, -5.86534614e-01,  3.85672897e-01, -3.24036280e-01,
       -4.24500169e-01, -

In [14]:
rewards[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [7]:
rewards_raw = pd.read_csv('/home/yes24/capstone/원진공유폴더/일요일에고생이많다/reward_10000000_0630.csv',nrows=100000)
df = pd.read_csv('/home/yes24/capstone/원진공유폴더/일요일에고생이많다/context_10000000_0630.csv',nrows=100000)

In [8]:
labels = rewards_raw['book_cluster'].astype('int8').values
df = df.iloc[:,1:]

In [9]:
df, labels = remove_underrepresented_classes(df, labels)
contexts = df.values

Unique classes and their ratio of total: {0: 0.12004, 1: 0.21227, 2: 0.14777, 3: 0.34609, 4: 0.17383}


In [10]:
sampled_vals = classification_to_bandit_problem(contexts, labels, num_actions)

In [11]:
contexts, rewards, (opt_rewards, opt_actions) = sampled_vals

In [12]:
dataset = np.hstack((contexts, rewards))
context_dim = contexts.shape[1]

In [20]:
hparams = tf.contrib.training.HParams(num_actions=num_actions)
hparams_linear = tf.contrib.training.HParams(num_actions=num_actions,
                                               context_dim=context_dim,
                                               a0=6,
                                               b0=6,
                                               lambda_prior=0.25,
                                               initial_pulls=1)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [21]:
algos = [LinearFullPosteriorSampling('LinFullPost', hparams_linear)]

In [24]:
algos[0].mu

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      

In [15]:
t_init = time.time()
results = run_contextual_bandit(context_dim, num_actions, dataset, algos)
_, h_rewards,h_betas = results

In [45]:
beta_df = pd.DataFrame(np.array(h_betas[-1])[0],columns=column_list)

In [46]:
beta_df.to_csv('beta_df_0630.csv',index=False)

In [40]:
column_list = ['constant','cat1.0',
 'cat2.0', 'cat3.0', 'cat4.0', 'cat5.0', 'cat7.0', 'cat8.0', 'cat9.0', 'cat10.0', 'cat11.0', 'cat12.0', 'cat13.0', 'cat14.0', 'cat15.0',
 'cat16.0', 'cat17.0', 'cat19.0', 'cat20.0', 'cat22.0', 'cat23.0', 'cat25.0', 'cat27.0', 'cat29.0', 'cat33.0', 'cat34.0', 'pub0.0',
 'pub1.0', 'pub2.0', 'pub3.0', 'pub4.0', 'pri0.0', 'pri1.0', 'pri2.0', 'pri3.0', 'index.1', 'gender_0.0', 'gender_1.0', 'g_age_0.0', 'g_age_1.0',
 'g_age_2.0', 'g_age_3.0', 'g_age_4.0', 'g_age_5.0', 'g_age_6.0', 'most_pref_1.0', 'most_pref_2.0', 'most_pref_3.0', 'most_pref_4.0',
 'most_pref_5.0', 'most_pref_7.0', 'most_pref_8.0', 'most_pref_9.0', 'most_pref_10.0', 'most_pref_11.0', 'most_pref_12.0', 'most_pref_13.0', 'most_pref_14.0',
 'most_pref_15.0', 'most_pref_16.0', 'most_pref_17.0', 'most_pref_19.0', 'most_pref_20.0', 'most_pref_22.0', 'most_pref_23.0', 'most_pref_25.0',
 'most_pref_27.0', 'most_pref_29.0', 'most_pref_33.0', 'most_pref_34.0', 'new_pref_0', 'new_pref_1', 'address_is_0', 'address_is_1']

In [44]:
display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 9291.318922519684 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =    98027.0.
---------------------------------------------------
Optimal total reward = 100000.0.
Frequency of optimal actions (action, frequency):
[[0, 12004], [1, 21227], [2, 14777], [3, 34609], [4, 17383]]
---------------------------------------------------
---------------------------------------------------


In [47]:
len(h_betas)

99995

In [49]:
dataset[0]

array([-0.14844216, -0.12543411,  2.97054671, -0.13740394, -0.21399855,
       -0.13698782, -0.14223637, -0.17772788, -0.14466874, -0.16221809,
       -0.05977222, -0.26137904, -0.24122575, -0.13322745, -0.16052188,
       -0.20353871, -0.19041715, -0.16058742, -0.21890968, -0.18310553,
       -0.07564761, -0.37457024, -0.28854689, -0.32935604, -0.19667349,
        1.94378174, -0.3655931 , -0.35041134, -0.54196679, -0.71202339,
       -0.41423575, -0.64042088,  1.24896025, -0.45585349, -1.73203349,
       -1.33826653,  1.33826653,  0.        ,  0.        , -0.01265012,
       -0.65571376,  0.96130795, -0.4622082 , -0.05849502, -0.1600953 ,
       -0.12390369,  2.50359143, -0.14599603, -0.21621229, -0.14603175,
       -0.15240492, -0.12514592, -0.14816045, -0.15804661, -0.03914517,
       -0.27674629, -0.2196861 , -0.11829245, -0.14340283, -0.1748027 ,
       -0.16383419, -0.11907229, -0.18158326, -0.15435153, -0.03703889,
       -0.45173546, -0.3026283 , -0.34351925, -0.11915865,  0.39

In [17]:
dataset.shape

(2458285, 76)

In [19]:
os.getcwd()

'/home/yes24/models/research/deep_contextual_bandits'